In [ ]:
import json
import re

import utils
import data_handler as dh

from collections import defaultdict
from itertools import product

import tensorflow as tf
import pandas as pd
import numpy as np

### Data settting

In [ ]:
data_path = '/home/donghyun/eye_writing_classification/v2_dataset/200_points_dataset/'

with open(data_path + 'eog_raw_numbers_200.json') as f:
  eog_raw_numbers = json.load(f)

with open(data_path + 'eog_katakana_200.json') as f:
  eog_katakana = json.load(f)

with open(data_path + 'reference_data_200.json') as f:
  reference_data = json.load(f)

In [ ]:
katakana_le = {'1':'10', '2':'11', '3':'12', '4':'13', '5':'14',
               '6':'15', '7':'16', '8':'17', '9':'18', '10':'19',
               '11':'20', '12':'21'}

all_data = eog_raw_numbers.copy()
for k in eog_katakana.keys():
    cnvt_key = katakana_le[k]
    all_data[cnvt_key] = eog_katakana[k]

all_ref = defaultdict()
all_ref['all'] = reference_data['numbers'].copy()
for k in reference_data['katakana'].keys():
    cnvt_key = katakana_le[k]
    all_ref['all'][cnvt_key] = reference_data['katakana'][k]


In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

best_perform_df = pd.read_csv(save_path + 'experiment2_vit_hyperparams.csv')

### Experiment

In [ ]:
class ViT_Config:
    split_ratio = 0.3
    ref_key = 'all'
    batch_size = 22            # fix : must be equaled with number of test pairs 
    n_batch = 100
    lr = 0.0005
    model_type = 'ViTBaseModel'
    ViT_params = {}
    epochs = 100

In [ ]:
top_k = 1

cfg_list = []
for i in range(top_k):
    cfg = ViT_Config()
    cfg.ViT_params = best_perform_df.iloc[i].to_dict()
    cfg.ViT_params['mlp_units'] = re.sub('[\[\]]','',cfg.ViT_params['mlp_units'])
    cfg.ViT_params['mlp_units'] = list(map(int,cfg.ViT_params['mlp_units'].split(',')))       # str to list
    cfg.ViT_params['batch_size'] = cfg.batch_size

    cfg_list.append(cfg)

In [ ]:
classes = all_data.keys()

result_dict = defaultdict(list)
times = 5
for t in range(times):
    print('times : ', t)
    for k in classes:
        zero_shot_cls = k

        # train without a class for zero shot learning
        model_list = []
        for i, cfg in enumerate(cfg_list):
            model, _, _, _ = utils.experiment(cfg, all_data, all_ref, zero_shot_cls=zero_shot_cls)
            model_list.append(model)

        # zero shot inference
        zero_shot_batch, zero_shot_targets = dh.get_zero_shot_batch(all_data, all_ref, ref_key='all', zero_shot_cls=zero_shot_cls)

        correct = 0
        i = 0
        for batch in zip(zero_shot_batch, zero_shot_targets):
            data, target = batch

            probs = np.zeros((target.shape[0],1))
            for m in model_list:
                probs += model.predict_on_batch(data)
            
            if np.argmax(probs) == np.argmax(target):
                correct += 1
            i+=1
        
        acc = (correct/i)*100
        result_dict[k].append(round(acc,4))

        print('class : {}\'s accuracy using ensemble : {:.4f}%'.format(k, acc))
        tf.keras.backend.clear_session()


### Save

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path + 'experiment4_zero_shot_results.json', 'w') as f:
    json.dump(dict(result_dict),f)

### Visualization

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path+'experiment4_zero_shot_results_ensemble.json') as f:
    ensemble_zero_shot_dict = json.load(f)

with open(save_path+'experiment4_zero_shot_results_best_param.json') as f:
    best_params_zero_shot_dict = json.load(f)
    

In [ ]:
for k in ensemble_zero_shot_dict.keys():
    print('zero shot class {}\'s accuracy : {}'.format(k, ensemble_zero_shot_dict[k]))

In [ ]:
for k in best_params_zero_shot_dict.keys():
    print('zero shot class {}\'s accuracy : {}'.format(k, best_params_zero_shot_dict[k]))